In [1]:
#Ensuring the current working directly has the environment file with API keys

import os

current_working_directory = os.getcwd()
print(f"Jupyter Notebook's current working directory: {current_working_directory}")

# Check for the .env file in this directory
env_file_path = os.path.join(current_working_directory, '.env')
if os.path.exists(env_file_path):
    print(f"'.env' file found at: {env_file_path}")
    # Also print its content to be absolutely sure
    with open(env_file_path, 'r') as f:
        print("\n--- .env file content ---")
        print(f.read())
        print("-------------------------\n")
else:
    print(f"ERROR: '.env' file NOT found at: {env_file_path}")
    print("Please ensure your .env file is in the current working directory or a parent directory.")

Jupyter Notebook's current working directory: /Users/hshekar/langgraph_test
'.env' file found at: /Users/hshekar/langgraph_test/.env

--- .env file content ---
WEATHER_API_KEY="5dba1f60bb7c49a09db130437251606"
TAVILY_API_KEY="Qoa3xtvNMIB9fYCE2ZemJ9GZiRVnJWKz"
TOGETHER_API_KEY="Ad08267a811875d5a525a7e2c39c3260ce347105eaa3c559f626aae9f23a6fb1"
api_key="219b45490dc00d37f87d78f11942bf48287964620e6850b1ab33f1923cd246c3"

-------------------------



In [2]:
# Load the environment variables
import os
from dotenv import load_dotenv
load_dotenv('.env')

WEATHER_API_KEY = os.environ['WEATHER_API_KEY']
TAVILY_API_KEY = os.environ['TAVILY_API_KEY']
TOGETHER_API_KEY = os.environ['TOGETHER_API_KEY']

In [3]:
# Import the required libraries and methods
import requests
from typing import List, Literal
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

In [4]:
# define the tools that are required for the agent 
@tool
def get_weather(query: str) -> list:
    """Search weatherapi to get the current weather"""
    endpoint = f"http://api.weatherapi.com/v1/current.json?key={WEATHER_API_KEY}&q={query}"
    response = requests.get(endpoint)
    data = response.json()

    if data.get("location"):
        return data
    else:
        return "Weather Data Not Found"

@tool
def search_web(query: str) -> list:
    """Search the web for a query"""
    tavily_search = TavilySearchResults(api_key=TAVILY_API_KEY, max_results=2, search_depth='advanced', max_tokens=1000)
    results = tavily_search.invoke(query)
    return results

In [10]:
#access API key via cosmos
from openai import OpenAI
llm = ChatOpenAI(base_url="https://aiplatform.dev51.cbf.dev.paypalinc.com/cosmosai/llm/v1",
    api_key="219b45490dc00d37f87d78f11942bf48287964620e6850b1ab33f1923cd246c3",
    model="llama31-8b")

tools = [search_web, get_weather]
llm_with_tools = llm.bind_tools(tools)

State maintains and updates the context or memory as the process advances and enabling each step to access relevant information from earlier steps for dynamic decision-making.

Nodes represent individual computation steps or tasks, performing specific functions like data processing, decision-making or system interactions.

Edges connect nodes, dictating the flow of computation and allowing conditional logic to guide the workflow based on the current state, Yes or No box

Stateful graph is central to its architecture, where each node represents a step in the computation. The graph maintains and updates a shared state as the process advances.

In [19]:
prompt = """
    Given only the tools at your disposal, mention tool calls for the following tasks:
    Do not change the query given for any search tasks
        1. What is the current weather in Trivandrum today
        2. Can you tell me about Kerala
        3. Why is the sky blue?
    """

results = llm_with_tools.invoke(prompt)

print(results.tool_calls)

query = "What is the current weather in Trivandrum today"
response = llm.invoke(query)
print(response.content)

[{'name': 'get_weather', 'args': {'query': 'Trivandrum'}, 'id': 'chatcmpl-tool-224a2f21e87949749c59cf9daab35862', 'type': 'tool_call'}, {'name': 'search_web', 'args': {'query': 'Kerala'}, 'id': 'chatcmpl-tool-fdb924f6c3d8420a9c16d3f5b0938853', 'type': 'tool_call'}, {'name': 'search_web', 'args': {'query': 'Why is the sky blue?'}, 'id': 'chatcmpl-tool-8b7960d5632940ec8b1ce12c616e3729', 'type': 'tool_call'}]
I can't verify the current weather in Trivandrum. However, I can suggest some options for you to find the current weather in Trivandrum. 

You can check the current weather in Trivandrum by visiting a weather website such as AccuWeather, Weather.com, or the India Meteorological Department (IMD) website. 

Alternatively, you can use a virtual assistant like Siri or Google Assistant to find the current weather in Trivandrum.


In [20]:
from langgraph.prebuilt import create_react_agent

# system prompt is used to inform the tools available to when to use each
system_prompt = """Act as a helpful assistant.
    Use the tools at your disposal to perform tasks as needed.
        - get_weather: whenever user asks get the weather of a place.
        - search_web: whenever user asks for information on current events or if you don't know the answer.
    Use the tools only if you don't know the answer.
    """

# we can initialize the agent using the llama3 model, tools, and system prompt.
agent = create_react_agent(model=llm, tools=tools)

# Let’s query the agent to see the result.
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

inputs = {"messages": [("user", "What is the current weather in Trivandrum today")]}

print_stream(agent.stream(inputs, stream_mode="values"))

================================ Human Message =================================

What is the current weather in Trivandrum today
================================== Ai Message ==================================
Tool Calls:
  get_weather (chatcmpl-tool-45789a86885f4a6aadd8ae4b1ab8adec)
 Call ID: chatcmpl-tool-45789a86885f4a6aadd8ae4b1ab8adec
  Args:
    query: Trivandrum
================================= Tool Message =================================
Name: get_weather

{"location": {"name": "Trivandrum", "region": "Kerala", "country": "India", "lat": 8.5069, "lon": 76.9569, "tz_id": "Asia/Kolkata", "localtime_epoch": 1750093964, "localtime": "2025-06-16 22:42"}, "current": {"last_updated_epoch": 1750093200, "last_updated": "2025-06-16 22:30", "temp_c": 25.3, "temp_f": 77.5, "is_day": 0, "condition": {"text": "Mist", "icon": "//cdn.weatherapi.com/weather/64x64/night/143.png", "code": 1030}, "wind_mph": 7.6, "wind_kph": 12.2, "wind_degree": 298, "wind_dir": "WNW", "pressure_mb": 1010.0, "

In [14]:
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, MessagesState, START, END

In [15]:
tools = [search_web, get_weather]
tool_node = ToolNode(tools)

In [16]:
def call_model(state: MessagesState):
    messages = state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

def call_tools(state: MessagesState) -> Literal["tools", END]:
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

In [17]:
# initialize the workflow from StateGraph
workflow = StateGraph(MessagesState)

# add a node named LLM, with call_model function. This node uses an LLM to make decisions based on the input given
workflow.add_node("LLM", call_model)

# Our workflow starts with the LLM node
workflow.add_edge(START, "LLM")

# Add a tools node
workflow.add_node("tools", tool_node)

# Add a conditional edge from LLM to call_tools function. It can go tools node or end depending on the output of the LLM. 
workflow.add_conditional_edges("LLM", call_tools)

# tools node sends the information back to the LLM
workflow.add_edge("tools", "LLM")

agent = workflow.compile()

In [18]:
for chunk in agent.stream(
    {"messages": [("user", "Will it rain in Trivandrum today?")]},
    stream_mode="values",):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

Will it rain in Trivandrum today?
================================== Ai Message ==================================
Tool Calls:
  get_weather (chatcmpl-tool-2a8b4e2497c1475cb2d55895412de5dc)
 Call ID: chatcmpl-tool-2a8b4e2497c1475cb2d55895412de5dc
  Args:
    query: Trivandrum
================================= Tool Message =================================
Name: get_weather

{"location": {"name": "Trivandrum", "region": "Kerala", "country": "India", "lat": 8.5069, "lon": 76.9569, "tz_id": "Asia/Kolkata", "localtime_epoch": 1750091887, "localtime": "2025-06-16 22:08"}, "current": {"last_updated_epoch": 1750091400, "last_updated": "2025-06-16 22:00", "temp_c": 25.0, "temp_f": 77.0, "is_day": 0, "condition": {"text": "Mist", "icon": "//cdn.weatherapi.com/weather/64x64/night/143.png", "code": 1030}, "wind_mph": 7.6, "wind_kph": 12.2, "wind_degree": 298, "wind_dir": "WNW", "pressure_mb": 1010.0, "pressure_in": 